<a href="https://colab.research.google.com/github/vtu22874-eng/QML-Tasks/blob/main/TASK_10_Implement_the_QAOA_algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
Aim: To implement the Quantum Approximate Optimization Algorithm (QAOA) using Qiskit and PyTorch to solve the Max-Cut problem, a classical NP-hard problem.
 Algorithm - QAOA Algorithm
1. Graph Construction
     Define adjacency matrix W.
     Build a NetworkX graph for visualization.
2. Classical Baseline
     Use brute-force enumeration to compute the optimal Max-Cut value (ground truth).
3. QAOA Circuit Construction
     Initialize qubits in | +〉  .
     Apply alternating cost and mixer unitaries for depth 𝑝.
     Use controlled-𝑍 rotation gates to implement 𝑍H𝑍H interactions.
4. Expectation Calculation
     Simulate circuit using Qiskit Aer statevector simulator.
     Compute expected cut value from measurement probabilities.
5. Hybrid Optimization
     Parameters 𝛾⃗,𝛽⃗ initialized randomly.
     Compute finite-difference gradients of expectation.
     Update parameters using PyTorch Adam optimizer.
6. Circuit Visualization
     Draw initial and optimized QAOA circuits using qiskit.visualization.

In [ ]:
# qaoa_toy_qiskit_torch.py
# Requirements:
#   pip install qiskit torch networkx numpy matplotlib

import os
import numpy as np
import networkx as nx
import torch
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector
import matplotlib

# Use Agg backend in headless environments so saving works without GUI
matplotlib.use(os.environ.get("MPLBACKEND", "Agg") or "Agg")
import matplotlib.pyplot as plt

# -------------------------
# Problem definition
# -------------------------
def make_graph():
    """
    Example 4-node graph used in many tutorials.
    Returns networkx graph and adjacency-weight matrix w (numpy array).
    """
    w = np.array([
        [0.0, 1.0, 1.0, 0.0],
        [1.0, 0.0, 1.0, 1.0],
        [1.0, 1.0, 0.0, 1.0],
        [0.0, 1.0, 1.0, 0.0]
    ])
    G = nx.from_numpy_array(w)
    return G, w

def objective_value(x, w):
    """
    MaxCut objective for bitstring x (array of 0/1).
    Sum of weights for edges crossing the cut: sum_{i<j} w[i,j] * (x_i != x_j)
    """
    n = len(x)
    val = 0.0
    for i in range(n):
        for j in range(i + 1, n):
            if w[i, j] != 0 and (x[i] != x[j]):
                val += w[i, j]
    return val

def brute_force_maxcut(w):
    """Brute-force search for best cut (only for small n)."""
    n = w.shape[0]
    best = -np.inf
    best_x = None
    for i in range(2**n):
        bits = np.array(list(map(int, np.binary_repr(i, width=n))))
        val = objective_value(bits, w)
        if val > best:
            best = val
            best_x = bits
    return best_x, best

# -------------------------
# Build QAOA circuit
# -------------------------
def qaoa_circuit(n_qubits, edges, gammas, betas):
    """
    Build QAOA circuit:
      - start in |+>^n
      - cost unitaries for each edge: exp(-i * gamma * w * Z_i Z_j)
      - mixer unitaries Rx(2*beta)
    edges: list of (i, j, weight) with i>j or i<j (we don't rely on ordering)
    gammas, betas: arrays/lists of length p
    """
    p = len(gammas)
    qc = QuantumCircuit(n_qubits)
    qc.h(range(n_qubits))  # create |+>^n

    for layer in range(p):
        gamma = float(gammas[layer])
        # cost layer
        for (i, j, w) in edges:
            if w == 0:
                continue
            # Implement exp(-i * gamma * w * Z_i Z_j)
            # Using CNOT - RZ(theta) - CNOT where theta = 2 * gamma * w
            theta = 2.0 * gamma * w
            qc.cx(i, j)
            qc.rz(theta, j)
            qc.cx(i, j)
        # mixer layer
        beta = float(betas[layer])
        for q in range(n_qubits):
            qc.rx(2.0 * beta, q)

    return qc

# -------------------------
# Expectation from statevector
# -------------------------
def expectation_from_statevector(statevector, w):
    """Compute expected MaxCut objective from a statevector (numpy array)."""
    probs = Statevector(statevector).probabilities_dict()
    exp_val = 0.0
    for bitstr, p in probs.items():
        # reverse bit order so qubit 0 -> leftmost? We choose convention that
        # bitstr returned by Statevector has qubit 0 as rightmost, so reverse it.
        bits = np.array([int(b) for b in bitstr[::-1]])
        val = objective_value(bits, w)
        exp_val += val * p
    return exp_val

# -------------------------
# QAOA + PyTorch classical loop
# -------------------------
def run_qaoa_with_pytorch(w, p=1, init_std=0.5, maxiter=100, lr=0.1, finite_diff_eps=1e-3):
    """
    Optimize QAOA parameters (gamma_1..gamma_p, beta_1..beta_p) using
    PyTorch optimizer with finite-difference gradients.
    Returns best found dictionary with keys: val, params (torch.Tensor), bitstring (np.array)
    """
    n = w.shape[0]
    # edges list (i,j,w) for i<j
    edges = [(i, j, float(w[i, j])) for i in range(n) for j in range(i + 1, n) if w[i, j] != 0.0]

    # params: concatenated [gamma_1..gamma_p, beta_1..beta_p]
    params = torch.nn.Parameter(torch.randn(2 * p, dtype=torch.double) * init_std)
    optimizer = torch.optim.Adam([params], lr=lr)

    best = {"val": -np.inf, "params": None, "bitstring": None}

    for it in range(maxiter):
        # detach numpy arrays for circuit building
        with torch.no_grad():
            flat = params.detach().cpu().numpy()
        gammas = flat[:p]
        betas = flat[p:]

        # build circuit and obtain statevector deterministically (no backend required)
        qc = qaoa_circuit(n, edges, gammas, betas)
        sv = Statevector.from_instruction(qc).data  # numpy array

        # expectation (we maximize expectation)
        exp_val = expectation_from_statevector(sv, w)
        loss = -float(exp_val)  # optimizer minimizes loss

        # keep best
        if exp_val > best["val"]:
            probs = Statevector(sv).probabilities_dict()
            most = max(probs.items(), key=lambda kv: kv[1])[0]
            bits = np.array([int(b) for b in most[::-1]])
            best.update({"val": exp_val, "params": params.detach().clone(), "bitstring": bits})

        # finite-difference gradient
        base = params.detach().cpu().numpy()
        grads = np.zeros_like(base, dtype=float)
        eps = finite_diff_eps

        for k in range(len(base)):
            plus = base.copy()
            minus = base.copy()
            plus[k] += eps
            minus[k] -= eps

            g_plus = _qaoa_expectation_with_params(plus, n, edges, w, p)
            g_minus = _qaoa_expectation_with_params(minus, n, edges, w, p)
            # derivative of loss = - expectation
            grad_k = - (g_plus - g_minus) / (2.0 * eps)
            grads[k] = grad_k

        # assign grads to params and step optimizer
        params.grad = torch.from_numpy(grads).to(dtype=torch.double)
        optimizer.step()
        optimizer.zero_grad()

        if (it % 10 == 0) or (it == maxiter - 1):
            print(f"Iter {it:03d}: expected cut = {exp_val:.6f}, loss = {loss:.6f}")

    return best

def _qaoa_expectation_with_params(flat_params, n, edges, w, p):
    """Helper: compute expected cut for given flat_params (no torch)."""
    gammas = flat_params[:p]
    betas = flat_params[p:]
    qc = qaoa_circuit(n, edges, gammas, betas)
    sv = Statevector.from_instruction(qc).data
    exp_val = expectation_from_statevector(sv, w)
    return exp_val

# -------------------------
# Circuit display helpers
# -------------------------
def show_circuit(qc: QuantumCircuit, filename: str = None, style: str = "mpl"):
    print("\n--- Quantum Circuit (text) ---")
    try:
        print(qc.draw(output="text"))
    except Exception as e:
        print("Failed to draw Quantum Circuit (text):", e)

    if style == "mpl":
        try:
            # qiskit circuit drawing to mpl returns a matplotlib.figure.Figure
            fig = qc.draw(output="mpl", interactive=False)
            fig.tight_layout()
            if filename:
                fig.savefig(filename, dpi=200, bbox_inches="tight")
                print(f"[Saved circuit figure to {filename}]")
            else:
                tempname = "qaoa_circuit.png"
                fig.savefig(tempname, dpi=200, bbox_inches="tight")
                print(f"[Saved circuit figure to {tempname}]")
            plt.close(fig)
        except Exception as e:
            print("Matplotlib drawing failed:", str(e))
            print("Fallback: shown text diagram above.")

def demo_display_initial_circuit(w, p=1, filename="qaoa_initial_circuit.png"):
    n = w.shape[0]
    gammas = np.random.randn(p) * 0.8
    betas = np.random.randn(p) * 0.8
    edges = [(i, j, float(w[i, j])) for i in range(n) for j in range(i + 1, n) if w[i, j] != 0.0]
    qc = qaoa_circuit(n, edges, gammas, betas)
    show_circuit(qc, filename=filename, style="mpl")

def demo_display_best_circuit(w, best_params, p=1, filename="qaoa_best_circuit.png"):
    n = w.shape[0]
    if isinstance(best_params, torch.Tensor):
        flat = best_params.detach().cpu().numpy()
    else:
        flat = np.array(best_params, dtype=float)
    gammas = flat[:p]
    betas = flat[p:]
    edges = [(i, j, float(w[i, j])) for i in range(n) for j in range(i + 1, n) if w[i, j] != 0.0]
    qc = qaoa_circuit(n, edges, gammas, betas)
    show_circuit(qc, filename=filename, style="mpl")

# -------------------------
# Run example
# -------------------------
if __name__ == "__main__":
    G, w = make_graph()
    print("Graph edges:", list(G.edges()))
    bf_x, bf_val = brute_force_maxcut(w)
    print("Brute-force best:", bf_x, "value:", bf_val)

    # show an initial example circuit (random parameters)
    demo_display_initial_circuit(w, p=1, filename="qaoa_initial_circuit.png")

    # run QAOA p=1 (toy)
    best = run_qaoa_with_pytorch(w, p=1, init_std=0.8, maxiter=80, lr=0.2, finite_diff_eps=1e-3)
    print("QAOA best expected value:", best["val"])
    print("Most-likely bitstring found:", best["bitstring"])

    # evaluate most-likely bitstring exactly
    if best["bitstring"] is not None:
        exact_val = objective_value(best["bitstring"], w)
        print("Exact value of that bitstring:", exact_val)

    # Display the optimized circuit using the best parameters (and save)
    if best["params"] is not None:
        demo_display_best_circuit(w, best["params"], p=1, filename="qaoa_best_circuit.png")
    else:
        print("No best params found to display.")


Graph edges: [(0, 1), (0, 2), (1, 2), (1, 3), (2, 3)]
Brute-force best: [0 1 1 0] value: 4.0

--- Quantum Circuit (text) ---
     ┌───┐                                                  ┌──────────────┐»
q_0: ┤ H ├──■───────────────────■────■───────────────────■──┤ Rx(-0.32233) ├»
     ├───┤┌─┴─┐┌─────────────┐┌─┴─┐  │                   │  └──────────────┘»
q_1: ┤ H ├┤ X ├┤ Rz(-1.9997) ├┤ X ├──┼───────────────────┼─────────■────────»
     ├───┤└───┘└─────────────┘└───┘┌─┴─┐┌─────────────┐┌─┴─┐     ┌─┴─┐      »
q_2: ┤ H ├─────────────────────────┤ X ├┤ Rz(-1.9997) ├┤ X ├─────┤ X ├──────»
     ├───┤                         └───┘└─────────────┘└───┘     └───┘      »
q_3: ┤ H ├──────────────────────────────────────────────────────────────────»
     └───┘                                                                  »
«                                                                  »
«q_0: ─────────────────────────────────────────────────────────────»
«                                  

In [ ]:
pip install qiskit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 63.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 75.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.4 MB/s eta 0:00:00


In [ ]:
pip install torch

In [ ]:
pip install networkx

In [ ]:
pip install numpy

In [ ]:
pip install matplotlib

In [ ]:
Result:
   Thus the program for implementing the qaoa algorithm were written, executed and verified successfully